In [ ]:
import glob
import numpy as np
import os
import pandas as pd
import pickle
from tqdm import tqdm, trange

In [ ]:
def get_vectors(filename):
    vectors = []
    with open(filename,"rb") as f:
        try:
            while True:
                x=pickle.load(f)
                vectors = x
        except EOFError:
            pass
    return vectors

In [ ]:
#Getting all the sentences for the dataset
def get_all_sentences():
    my_sentences = []
    my_labels = []
    
    file_name_regex = '../medical_data/train_data/cleaned_files/*.dat'
    file_prefix = '../medical_data/train_data/cleaned_files/'
    l = [file_prefix + os.path.basename(x) for x in glob.glob(file_name_regex)]

    for i in l:
        if 'label_dicts' not in i:
            print("Vvctors for file")
            g = get_vectors(i)
            print("g",g)
            my_sentences+=g[1]
            my_labels+=g[2]
    return my_sentences,my_labels


#Getting all the sentences
myt_sentences,myt_labels = get_all_sentences()
print(myt_sentences[2])

print(myt_labels,len(myt_labels))

In [ ]:
vec = get_vectors("../medical_data/train_data/cleaned_files/879492218_YC.dat")
my_sentences = vec[1]
print("myt",myt_sentences[0:30])
print("############################################################")
print("my",my_sentences[0:30])
for x in myt_sentences:
    try:
        print("TYPE",type(x),x)
        b = " ".join(x)
        print(b)
    except:
        print("ERROR")
        print("x",x)
new_sentences = [" ".join(x) for x in myt_sentences]
#new_sentences = [" ".join(x) for x in my_sentences]
print("############################################################")
print("newt_sen",newt_sentences)

tag_map = {'problem_b':0,
           'problem_i':1,
          'problem_e':2,
          'problem_s':3,
          'test_b':4,
          'test_i':5,
          'test_e':6,
          'test_s':7,
          'treatment_b':8,
          'treatment_i':9,
          'treatment_e':10,
          'treatment_s':11,
          'o':12}
#print(vec[2])

reverse_tag_map = dict((v,k) for k,v in tag_map.items())


#get all the labels



new_labels = [[reverse_tag_map[j] for j in i] for i in myt_labels]
print(new_labels)

In [ ]:
data = pd.read_csv("entity-annotated-corpus/ner_dataset.csv",encoding="latin1").fillna(method="ffill") #ffill fills the NAN to take the previous cell value, in this case Sentence1
data.head(25)
#Can we get our data in this format??? looks like a good representation

In [ ]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
getter = SentenceGetter(data)

In [ ]:
# placeholder for our sentences 
#my_sentences = [" ".join([s for s in sent]) for sent in my_sentences]
new_sentences = [" ".join(x) for x in my_sentences]

print(new_sentences[4])
print(new_sentences[5])
print(new_sentences[11])
print(new_sentences[51])

sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[2]
print(sentences[4])
print(sentences[5])
print(sentences[6])

In [ ]:
# Here get the labels in the remapped BIO tag form
labels = [[s[2] for s in sent] for sent in getter.sentences]

In [ ]:
labels

In [ ]:
tag_vals = list(set(data["Tag"].values))
print (tag_vals)
tag2idx = {t:i for i,t in enumerate(tag_vals)}


tagvals = list(tag_map.keys())
print(tagvals)

## Preparing the sentences and labels for use with pytorch and BERT, before fine-tuning the model

In [ ]:
import torch

#Adam optimizer
from torch.optim import Adam

#TensorDataset - dataset wrapping tensors, Each sample will be retrieved by indexing tensors alg the first dimension
#DataLoader - combines a dataset and a sampler, provides a iterable over the dataset
#RandomSampler - sample randomly from shuffled dataset(without replc), no_of_samples(user based for with replac)
#SequentialSampler - Samples elements sequentially, always in the same order.
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig,BertModel
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [ ]:
MAX_LEN = 75
bs = 32

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
#Define BertTokenizer( base-BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case = True)

In [ ]:
#tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#Tokenize all the sentences in the form of BERT based representation ( ## word piece split)
tokenized_texts = [tokenizer.tokenize(sent) for sent in new_sentences]

In [ ]:
#print(sentences[4])
print(tokenized_texts[100])

In [ ]:

x = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
#print(x,len(x))

#Cut and pad the token and label sequences to our desired length
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen = MAX_LEN, value = tag_map["o"],padding="post",
                          dtype = "long",truncating = "post")

# tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
#                      maxlen = MAX_LEN,value = tag2idx["O"],padding = "post",
#                     dtype="long",truncating="post")
tags = pad_sequences([[tag_map.get(l) for l in lab] for lab in new_labels],
                     maxlen = MAX_LEN,value = tag_map["o"],padding = "post",
                    dtype="long",truncating="post")
print(tags.shape)

In [ ]:
input_ids.shape

In [ ]:
#BERT supported attention mask , similar to masking in keras
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [ ]:
#split the dataset to use 10% to validate the model
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)
print(tr_inputs.shape)
print(val_inputs.shape)

In [ ]:
#converting the dataset to tensors for pytorch
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
print(tr_inputs,tr_inputs.shape)

In [ ]:
#Define the Dataloaders

#Shuffling the training data
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

#Passing the test data sequentially
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
#config = BertConfig(output_hidden_states=True)
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag_map))

## Trying out Word embeddings example ( getting word vectors for contextual words)

In [ ]:
new_model = BertModel.from_pretrained('bert-base-uncased')

# Put the model in "evaluation" mode, meaning feed-forward operation.
new_model.eval()

In [ ]:
with torch.no_grad():
    encoded_layers, _ = new_model(tr_inputs, tr_tags)
    
print ("Number of layers:", len(encoded_layers))
layer_i = 0

print ("Number of batches:", len(encoded_layers[layer_i]))
batch_i = 0

print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))

In [ ]:
#Setting up the Adam optimizer by passing the parameters that should update. Add weight decay as regularization
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)
print(optimizer)

In [ ]:
#Metrics to be tracked during training
from seqeval.metrics import f1_score

def flat_accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## Fine tuning the model

In [ ]:
epochs = 2
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    print("Post train loss")
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        print("logits",logits)
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    
    print("nb_eval_steps",nb_eval_steps)
    
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    print("predictions",predictions)
    print("true labels",true_labels)

    pred_tags = [tagvals[p_i] for p in predictions for p_i in p]
    valid_tags = [tagvals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

In [ ]:
#Evaluation

model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [tagvals[p_i] for p in predictions for p_i in p]
valid_tags = [tagvals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))